---

# CSCI 3202, Fall 2022
# Homework 4
# Due: Friday September 30, 2022 at 6:00 PM

<br> 

### Your name: Ty VanEssen

<br> 

---

Some useful packages and libraries:




In [1]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part II, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right length, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>
$$ \frac{(1+number of heads)}{(2+number of flips)} $$

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [2]:
## Your code goes here.
## Your code needs to display the first 10 lines of your emission and transition tables, 
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname
## Your code also needs to be structured as an HMM class
import string

class HMM ():
    all_characters = ["_"] + list(string.ascii_lowercase)
    emission_cpt = pd.DataFrame(1, all_characters, all_characters)
    transition_cpt = pd.DataFrame(1, all_characters, all_characters)
    initial_distribution = {key : 1 for key in all_characters}
    
    def __init__(self, training='typos20.data'):
        self.data_file = pd.read_csv(training, sep=" ", header=0, names=("state","observation"))
        self.emission_cpt = HMM.emission_cpt
        self.transition_cpt = HMM.transition_cpt
        self.initial_distribution = HMM.initial_distribution

        for line in self.data_file.itertuples():
            self.emission_cpt.loc[ line.state, line.observation ] += 1
            if line.Index != 0:
                self.transition_cpt.loc[line.state, self.data_file.loc[line.Index - 1, "state"] ] += 1
            self.initial_distribution[line.state] += 1

        # Laplace smoothing already applied by seeing the values initially as 1
        self.emission_cpt = self.emission_cpt.apply(func=lambda column: column / sum(column), axis=1)

        self.transition_cpt = self.transition_cpt.apply(func=lambda column: column / sum(column), axis=1)
        # self.transition_cpt = pd.DataFrame(map( lambda column:  self.transition_cpt[column] / (sum(self.transition_cpt[column])), self.transition_cpt.columns))

        initial_distribution_constant = sum(self.initial_distribution.values())
        self.initial_distribution = { item[0]: item[1] / initial_distribution_constant for item in self.initial_distribution.items()}

model = HMM()
print(model.emission_cpt.head(10))
print(model.transition_cpt.head(10))
file = open('Assignment8Output_VanEssen_Tyler', 'w')
print(model.emission_cpt, file=file)
print(model.transition_cpt, file=file)
file.close()

          _         a         b         c         d         e         f  \
_  0.999044  0.000037  0.000037  0.000037  0.000037  0.000037  0.000037   
a  0.000101  0.794792  0.000101  0.000101  0.000101  0.000101  0.000101   
b  0.000507  0.000507  0.798784  0.000507  0.000507  0.000507  0.000507   
c  0.000220  0.000220  0.000220  0.793521  0.044954  0.000220  0.052887   
d  0.000233  0.000233  0.000233  0.041007  0.801025  0.040541  0.033784   
e  0.000059  0.000059  0.000059  0.000059  0.066348  0.800662  0.000059   
f  0.000319  0.000319  0.000319  0.033536  0.030981  0.000319  0.802619   
g  0.000388  0.000388  0.041925  0.000388  0.000388  0.000388  0.041537   
h  0.000159  0.000159  0.043699  0.000159  0.000159  0.000159  0.000159   
i  0.000098  0.000098  0.000098  0.000098  0.000098  0.000098  0.000098   

          g         h         i  ...         q         r         s         t  \
_  0.000037  0.000037  0.000037  ...  0.000037  0.000037  0.000037  0.000037   
a  0.000101  0


## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Canvas called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [8]:
## Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
## You also need to display the error rate for your algorithm.

# Referenced from https://en.wikipedia.org/wiki/Viterbi_algorithm
import sys


def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0] [st] = {"prob": np.log(start_p[st]) + np.log(emit_p.loc[st, obs[0]]), "prev": None}
    # Run through the length of the document
    for t in range(1, len(obs)):
        V.append({})
        for X_t in states:
            # P(X_t | X_t-1)*V_t-1*X_t-1
            # For every new state that we could transition to, what is the most likely state it could come from
            max_tr_prob, max_state = max(
                    (V[t - 1] [test_state] ["prob"] + np.log(trans_p.loc[X_t, test_state]), test_state)
                    for test_state in states
                )

            # *P(X_t | E_t)
            # The odds of getting to that new state * the probability of seeing the evidence we did.
            max_prob = max_tr_prob + np.log(emit_p.loc[X_t, obs[t]])
            V[t] [X_t] = {"prob": max_prob, "prev": max_state}

    opt = []
    max_prob = -sys.float_info.max
    best_st = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backfacing links
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1] [previous] ["prev"])
        previous = V[t + 1] [previous] ["prev"]

    return V, opt

In [9]:
model = HMM() if not model else model
test_file_name = 'typos20Test.data'
observations = pd.read_csv(test_file_name, sep=" ", header=0, names=("state","observation"))
V, opt = viterbi(observations.observation, model.all_characters, model.initial_distribution, model.transition_cpt, model.emission_cpt)

In [ ]:
# print("Viberti algorithm generates:\n", "".join(opt).replace('_', ' '))
test_file = open(test_file_name)
test_file.readline() # Throwaway the initial ..

def line_matches(inputline:str, myline:str):
    if inputline[2] == myline:
        return 1
    else:
        return 0

line_count = 0
correct = 0
for line in test_file:
    correct += line_matches(line, opt[line_count])
    line_count += 1

print("Error Rate %:", (1 - (correct / len(opt))) * 100)

Error Rate %: 10.930023923444976


## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 
Purpose should be clear. It doesn't need to be long, but it should be clear.
### Procedure:
##### Transition Probabilities
In this section explain the way you have generated the Transition probabilities and the data structure you used.
##### Emission Probabilities
In this section explain the way you have generated the Emission probabilities and the data structure you used.
##### Viterbi Algorithm
In this section explain about the algorithm and the data structure you used for storing the values in the matrix with its back pointer.
### Data
Explain about the data you used, including the data included in the data set and any pre-processing you did to the data.
### Results
Explain your observations, including the sentences that you tested and their corresponding outputs. Were there any sentences where your algorithm did not work?


### Purpose: 
The purpose of this HMM is to detect mistyped characters in relation to the characters before and after it.
### Procedure:
##### Transition Probabilities
Transition probabilites were generated by scanning the test data and tracking for each character what the previous charater was. By this we could then average the "next" transitioned to characters for each. A +1 ofset was included at the start to enable la-place smoothing.
##### Emission Probabilities
Emission probabilites were generated by scanning the test data and comparing the correct character to the emitted character. By this was then averaged out by seeing what characters appeared when a given character showed up. A +1 ofset was included at the start to enable la-place smoothing.
##### Viterbi Algorithm
The viterbi algorithm takes the initial probability of seeing a character as a base. Then, iteratively it computes what the highest odd character would be next by multipying it against all possible transitions to get there and the odds of observing the emission. It keeps track of all possible states at each time step and follows reverse links to know which character probably came previously and thus has the best odds of being the correct translation.
### Data
I used the data at base, it included _ instead of spaces, but I allowed this to be their representation in the model. Addionally the leading ".." was stripped on input by pandas.
### Results
I tested against the entire dataset. This assignment was submitted late since I wanted to ensure that I had full comprension of the model. It matches the provided answer text. I found that the emission probabilities mattered most on mistyped charaters and helped to refine the transition probabilites guesses at possible next characters. Previous iterations of the model found it semi-possible with a high error rate to transcribe with a malformed transition table OR emission table, the worst outputs were with incorrect both. The best output, obviously is when the tables were correct.